<img src="https://www.couponskiss.com/wp-content/uploads/2017/12/Dobot-Logo-200x200.png" align='left'>
<img src="https://www.eptecstore.com/wp-content/uploads/2018/03/S70-0480P01WL.jpg" width=50%>

This notebook explores the use of the Dobot Magician Robot

In [ ]:
#!pip install pydobot

In [ ]:
#!git clone https://github.com/luismesas/pydobot.git

In [ ]:
import pydobot

In [ ]:
import time
from glob import glob

from pydobot import Dobot

available_ports = glob('/dev/cu*usb*')  # mask for OSX Dobot port
if len(available_ports) == 0:
    print('no port found for Dobot Magician')
    exit(1)

device = Dobot(port=available_ports[0])

In [ ]:
time.sleep(0.5)
device.speed(100)
device.go(0.0, 0.0, 0.0)

In [ ]:
device.speed(10)
device.go(250.0, 0.0, 0.0)
time.sleep(2)

In [ ]:
device.goj(0,0,0)

In [ ]:
device.goj(-45,0,0)

###### 

In [ ]:
# DOBOT Controler

%matplotlib inline
import numpy as np
import math
import matplotlib.pylab as plt
from ipywidgets import interact
from mpl_toolkits.mplot3d import Axes3D


def Robot_Simulator(j1=0,j2=-0,j3=0):
    device.goj(j1,j2,j3)
    
target = interact(Robot_Simulator, j1=(-90,90), j2=(-5,90), j3=(-15,50)); 

In [ ]:
device.j1

In [ ]:
device.j2

In [ ]:
device.j3

In [ ]:
device.close()

In [ ]:
import numpy as np
import math
q1 = 0
d1 = 1

In [ ]:
j1 = np.matrix([[math.sin(q1), -math.cos(q1), 0, 0], [math.cos(q1), math.sin(q1), 0, 0], [0,0, 1, 0, d1]])
j1


In [ ]:
j2 = np.matrix([[math.sin(q1), -math.cos(q1), 0, 0], [math.cos(q1), math.sin(q1), 0, 0], [0,0, 1, 0, d1]])
j2


In [ ]:
# DOBOT SIMULATOR

%matplotlib inline
import numpy as np
import math
import matplotlib.pylab as plt
from ipywidgets import interact
from mpl_toolkits.mplot3d import Axes3D


def Robot_Simulator(j1=0,j2=-0,j3=0):
    
    #Convert input joint angles to radians
    q1 = j1/180  * np.pi
    q2 = j2/180  * np.pi
    q3 = j3/180  * np.pi
    
    d1 = 2 #Base to second joint
    d2 = 4 #Second to third joint
    d3 = 4 #Third to end effector
    dy = 1 #end effector y direction
    dz = 2 #end effector z direction
        
    #Make the rigid-pen end effector
    p = np.matrix([[0,dy, 3*dz/4, 1],
                   [0,dy, -dz/4, 1],
                   [0,dy,0,1], 
                   [0,0,0,1]]).T
    

    #From third to third (There is no motor here, we just need to subtract the angle)
    J4 = np.matrix([[1, 0, 0, 0],
                    [0, np.cos(-q3), -np.sin(-q3), d3 ], 
                    [0,  np.sin(-q3), np.cos(-q3), 0 ], 
                    [0, 0, 0, 1 ]])
    p = np.concatenate( ( J4*p, np.matrix([0,0,0,1]).T), axis=1 )

    #From Second to third (This is the tricky part because we need to subtract the secon angle due to joint coupling)
    J3 = np.matrix([[1, 0, 0, 0],
                    [0, np.cos(q3-q2), -np.sin(q3-q2), 0 ], 
                    [0,  np.sin(q3-q2), np.cos(q3-q2), -d2 ],  
                    [0, 0, 0, 1 ]])
    p = np.concatenate( ( J3*p, np.matrix([0,0,0,1]).T), axis=1 )
    
    #From first to Second Joint
    J2 = np.matrix([[1, 0, 0, 0],
                    [0, np.cos(q2), -np.sin(q2), 0 ], 
                    [0, np.sin(q2),  np.cos(q2), -d1 ], 
                    [0, 0, 0, 1 ]])
    p = np.concatenate((J2*p, np.matrix([0,0,0,1]).T), axis=1 )
    
    #From base to first joint
    J1 = np.matrix([[np.cos(q1), -np.sin(q1), 0, 0 ], 
                    [np.sin(q1), np.cos(q1), 0, 0 ], 
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])
    p = np.concatenate((J1*p, np.matrix([0,0,0,1]).T), axis=1 )
                    
    
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(p[0,:].tolist()[0],(p[1,:]).tolist()[0], (-p[2,:]).tolist()[0], s=20, facecolors='none', edgecolors='r')
    ax.scatter(0,0,0, s=20, facecolors='r', edgecolors='r')
    ax.plot(p[0,:].tolist()[0],(p[1,:]).tolist()[0], (-p[2,:]).tolist()[0])
    ax.set_xlim([-10.5,10.5])
    ax.set_ylim([-10.5,10.5])
    ax.set_zlim([0,10.5])
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')    
    ax.set_zlabel('z-axis') 

    plt.show()
    
target = interact(Robot_Simulator, j1=(-180,180), j2=(-180,180), j3=(-180,180)); 